<a href="https://colab.research.google.com/github/giramos/Machine-Learning/blob/master/EjerciciosPreparatorios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparación Cuestionario 💣
Ejercicios propuestos para preparación a un futuro cuestionario. Son ejemplos de lo visto en prácticas

## 1. Ejemplo
En este ejercicio se proporciona el conjunto de datos de
entrenamiento datos_entrenamiento_ssii.csv sobre el cual
ya se han aplicado todas las fases de preprocesamiento
necesarias. Utiliza este conjunto de datos para entrenar
modelos usando el algoritmo de los vecinos de cercanos
probando distintas combinaciones de números de vecinos
y distancias. En concreto, prueba todas las posibles
combinaciones formadas con los números de vecinos 3, 7,
y 11 y las distancias Manhattan y Euclídea. Para cada
combinación de parámetros utiliza GridSearchCV para
obtener los resultados de la accuracy en validación cruzada
empleando 10 cajas. Observa los resultados del modelo
entrenado con 3 vecinos y la distancia Manhattan: ¿Cuál es
el valor de la accuracy obtenida al validar el modelo
utilizando la caja con id 8 y el resto de cajas para entrenar?

**ChatGPT Solución**

In [73]:
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

# Cargar los datos (se asume que el dataset está preprocesado y listo para usar)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')

# Separar las variables predictoras (X) y las etiquetas (y)
#X = df.drop(columns=["Etiqueta"])  # Reemplaza "Etiqueta" con el nombre de la columna de etiquetas
#y = df["Etiqueta"]
X = df.drop(columns=["Clase"])  # Reemplaza "Etiqueta" con el nombre de la columna de etiquetas
y = df["Clase"]

# Configuración de parámetros para KNN
param_grid = {
    "n_neighbors": [3, 7, 11],
    "metric": ["manhattan", "euclidean"]
}

# Crear el modelo
knn = KNeighborsClassifier()

# Configurar GridSearchCV con validación cruzada de 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
grid_search = GridSearchCV(knn, param_grid, cv=cv, scoring="accuracy", return_train_score=True)

# Entrenar y realizar validación cruzada
grid_search.fit(X, y)

# Mostrar los resultados generales del GridSearchCV
results = pd.DataFrame(grid_search.cv_results_)
print("Resultados de GridSearchCV:")
print(results[["param_n_neighbors", "param_metric", "mean_test_score", "std_test_score"]])

# Analizar el modelo con 3 vecinos y distancia Manhattan
specific_model = results[
    (results["param_n_neighbors"] == 3) & (results["param_metric"] == "manhattan")
]

# Mostrar las métricas de validación cruzada
print("\nResultados del modelo con 3 vecinos y distancia Manhattan:")
print(specific_model[["mean_test_score", "std_test_score"]])

# Extraer resultados por fold usando StratifiedKFold
cv_split = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
accuracy_fold = []

for train_index, test_index in cv_split.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Entrenar con los datos de entrenamiento
    model = KNeighborsClassifier(n_neighbors=3, metric="manhattan")
    model.fit(X_train, y_train)

    # Evaluar con los datos de test (fold específico)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_fold.append(accuracy)

# Mostrar el resultado del fold con ID 8
print(f"\nAccuracy en el fold 8: {accuracy_fold[7]:.4f}")  # Recordar que los índices comienzan en 0


Resultados de GridSearchCV:
   param_n_neighbors param_metric  mean_test_score  std_test_score
0                  3    manhattan         0.823077        0.070115
1                  7    manhattan         0.879487        0.066974
2                 11    manhattan         0.807692        0.083383
3                  3    euclidean         0.864103        0.069739
4                  7    euclidean         0.871154        0.041390
5                 11    euclidean         0.855769        0.061968

Resultados del modelo con 3 vecinos y distancia Manhattan:
   mean_test_score  std_test_score
0         0.823077        0.070115

Accuracy en el fold 8: 0.8333


**Copilot Solución**

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# 1. Carga el conjunto de datos y define los parámetros:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')
X = data.drop('Clase', axis=1)
y = data['Clase']

param_grid = {
    'n_neighbors': [3, 7, 11],  # Valores de vecinos
    'p': [1, 2]  # Distancias Manhattan (1) y Euclídea (2)
}

# 2. Divide el conjunto de datos en entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Crea un modelo KNN e inicializa GridSearchCV:
knn_model = KNeighborsClassifier()
grid_search = GridSearchCV(knn_model, param_grid, cv=10, scoring='accuracy')

# 4. Entrena el modelo y observa los resultados:
grid_search.fit(X_train, y_train.values.ravel())

# 5. Encuentra el valor de la accuracy para la caja 8:
results_df = pd.DataFrame(grid_search.cv_results_)
accuracy_split8 = results_df[
    (results_df['param_n_neighbors'] == 3) & (results_df['param_p'] == 1)
]['split8_test_score'].values[0]

print(f"La accuracy en la caja 8 con 3 vecinos y distancia Manhattan es: {accuracy_split8:.2f}")

La accuracy en la caja 8 con 3 vecinos y distancia Manhattan es: 0.80


**NoteBook Solución**


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1. Carga el conjunto de datos y define los parámetros:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')
X = data.drop('Clase', axis=1)
y = data['Clase']

param_grid = {
    'n_neighbors': range(1,4),  # Ejemplo de valores válidos
    'p': [1, 4, 5]  # Distancias Manhattan y Euclídea
}

# 2. Divide el conjunto de datos en entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Crea un modelo KNN e inicializa GridSearchCV:
knn_model = KNeighborsClassifier()
grid_search = GridSearchCV(knn_model, param_grid, cv=10, scoring='accuracy')

# 4. Entrena el modelo y observa los resultados:
grid_search.fit(X_train, y_train.values.ravel())
print(grid_search.cv_results_)

# 5. Encuentra el valor de la accuracy para la caja 8:
results_df = pd.DataFrame(grid_search.cv_results_)
accuracy_split8 = results_df[
    (results_df['param_n_neighbors'] == 3) & (results_df['param_p'] == 1)
]['split8_test_score'].values

print(f"La accuracy en la caja 8 con 3 vecinos y distancia Manhattan es: {accuracy_split8}")



{'mean_fit_time': array([0.00200284, 0.00187595, 0.00195441, 0.0023396 , 0.00226429,
       0.00199742, 0.0020313 , 0.00209563, 0.00224156]), 'std_fit_time': array([1.23204482e-04, 5.60315162e-05, 2.01912054e-04, 6.32475286e-04,
       7.67925313e-04, 1.44497584e-04, 1.48635865e-04, 2.90543420e-04,
       4.16299783e-04]), 'mean_score_time': array([0.00356731, 0.00396011, 0.00402815, 0.00450921, 0.00438499,
       0.00408599, 0.00379322, 0.00434594, 0.00458481]), 'std_score_time': array([0.00017291, 0.00014073, 0.00045283, 0.00107665, 0.00073059,
       0.00019599, 0.00025812, 0.00058594, 0.00084898]), 'param_n_neighbors': masked_array(data=[1, 1, 1, 2, 2, 2, 3, 3, 3],
             mask=[False, False, False, False, False, False, False, False,
                   False],
       fill_value=999999), 'param_p': masked_array(data=[1, 4, 5, 1, 4, 5, 1, 4, 5],
             mask=[False, False, False, False, False, False, False, False,
                   False],
       fill_value=999999), 'param

**Gemini solución**

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler  # Para escalar los datos si es necesario

# Cargar los datos
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')

# Separar las características (X) y la variable objetivo (y)
X = data.drop('Clase', axis=1)
y = data['Clase']

# Escalar los datos (opcional, si las características tienen escalas muy diferentes)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Definir los parámetros a probar
param_grid = {
    'n_neighbors': [3, 7, 11],
    'metric': ['manhattan', 'euclidean']
}

# Fijar la semilla aleatoria para reproducibilidad
random_state = 42

# Mezclar los datos aleatoriamente
data = data.sample(frac=1, random_state=random_state)

# Dividir el conjunto de datos en entrenamiento y prueba (opcional, si quieres evaluar en un conjunto de prueba independiente)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)

# Crear el modelo KNN
knn = KNeighborsClassifier()

# Utilizar GridSearchCV con 10-fold cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los resultados
results = pd.DataFrame(grid_search.cv_results_)

# Encontrar la accuracy para 3 vecinos y distancia Manhattan en la caja 8
accuracy_caja8 = results[(results['param_n_neighbors'] == 3) &
                         (results['param_metric'] == 'manhattan')]['split8_test_score'].values[0]

print(f"Accuracy para 3 vecinos y distancia Manhattan en la caja 8: {accuracy_caja8}")

# Evaluar en el conjunto de prueba (opcional)
if X_test is not None:
    y_pred = grid_search.best_estimator_.predict(X_test)
    print(f"Accuracy en el conjunto de prueba: {accuracy_score(y_test, y_pred)}")

Accuracy para 3 vecinos y distancia Manhattan en la caja 8: 0.8
Accuracy en el conjunto de prueba: 0.8


**Solución Otra**

In [7]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')
X = df.drop('Clase', axis = 1)
y = df['Clase']
param_grid = {
    'n_neighbors': [3],
    'p': [1]
}
model = KNeighborsClassifier()
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.2, random_state=42, stratify=y)
grid_search = GridSearchCV(model, param_grid, cv=10,
scoring='accuracy')
grid_search.fit(X, y.values.ravel())
print(grid_search.cv_results_)


{'mean_fit_time': array([0.00638545]), 'std_fit_time': array([0.00298493]), 'mean_score_time': array([0.01353161]), 'std_score_time': array([0.00604712]), 'param_n_neighbors': masked_array(data=[3],
             mask=[False],
       fill_value=999999), 'param_p': masked_array(data=[1],
             mask=[False],
       fill_value=999999), 'params': [{'n_neighbors': 3, 'p': 1}], 'split0_test_score': array([0.76923077]), 'split1_test_score': array([0.69230769]), 'split2_test_score': array([0.69230769]), 'split3_test_score': array([0.84615385]), 'split4_test_score': array([1.]), 'split5_test_score': array([1.]), 'split6_test_score': array([0.75]), 'split7_test_score': array([0.83333333]), 'split8_test_score': array([1.]), 'split9_test_score': array([0.66666667]), 'mean_test_score': array([0.825]), 'std_test_score': array([0.12704766]), 'rank_test_score': array([1], dtype=int32)}


## 2. Ejemplo
En este ejercicio se proporciona el conjunto de datos de
entrenamiento datos_entrenamiento_ssii.csv sobre el cual
ya se han aplicado todas las fases de preprocesamiento
necesarias. Utiliza este conjunto de datos para entrenar
modelos usando el algoritmo de los vecinos de cercanos
probando distintas combinaciones de números de vecinos
y distancias. En concreto, prueba todas las posibles
combinaciones formadas con los números de vecinos 3, 7,
y 11 y las distancias Manhattan y Euclídea. Para cada
combinación de parámetros utiliza GridSearchCV para
obtener los resultados de la accuracy en validación cruzada
empleando 10 cajas. Observa los resultados del modelo
entrenado con 7 vecinos y la distancia Manhattan: ¿Cuál es
el valor de la accuracy obtenida al validar el modelo
utilizando la caja con id 4 y el resto de cajas para entrenar?

**Copilot Solución**

In [59]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Cargar los datos
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')

# Separar las características de la etiqueta
X = df.drop('Clase', axis=1)
y = df['Clase']

# Definir los parámetros para GridSearchCV
param_grid = {
    'n_neighbors': [3, 7, 11],
    'p': [1, 2]  # 1 para Manhattan, 2 para Euclídea
}

# Crear el modelo KNN
model = KNeighborsClassifier()

# Configurar GridSearchCV con validación cruzada de 10 folds
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')

# Entrenar el modelo
grid_search.fit(X, y)

# Obtener los resultados de la validación cruzada
results = grid_search.cv_results_

# Convert the results to a pandas DataFrame for easier indexing
results_df = pd.DataFrame(results)

# Obtener el valor de la accuracy para el modelo con 7 vecinos y distancia Manhattan en la caja con id 4
split4_test_score = results_df.loc[
    (results_df['param_n_neighbors'] == 7) & (results_df['param_p'] == 1),
    'split4_test_score'
].values[0]

print(f"Accuracy en la caja con id 4 para 7 vecinos y distancia Manhattan: {split4_test_score}")

Accuracy en la caja con id 4 para 7 vecinos y distancia Manhattan: 1.0


**Solución Gemini**

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Cargar los datos
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')

# Separar las características (X) y la variable objetivo (y)
X = data.drop('Clase', axis=1)  # Suponiendo que la columna objetivo se llama 'Clase'
y = data['Clase']

# Escalar los datos (opcional, si las características tienen escalas muy diferentes)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Definir los parámetros a probar
param_grid = {
    'n_neighbors': [3, 7, 11],
    'metric': ['manhattan', 'euclidean']
}

# Fijar la semilla aleatoria para reproducibilidad
random_state = 42

# Mezclar los datos aleatoriamente
data = data.sample(frac=1, random_state=random_state)

# Crear el modelo KNN
knn = KNeighborsClassifier()

# Utilizar GridSearchCV con 10-fold cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X, y)

# Obtener los resultados
results = pd.DataFrame(grid_search.cv_results_)

# Encontrar la accuracy para 7 vecinos y distancia Manhattan en la caja 4
accuracy_caja4 = results[(results['param_n_neighbors'] == 7) &
                         (results['param_metric'] == 'manhattan')]['split4_test_score'].values[0]  # split3 corresponde a la caja 4

print(f"Accuracy para 7 vecinos y distancia Manhattan en la caja 4: {accuracy_caja4}")

Accuracy para 7 vecinos y distancia Manhattan en la caja 4: 1.0


**Solución notebook**

In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# 1. Carga el conjunto de datos y define los parámetros:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')

# 2. Separar las características de la etiqueta
X = df.drop('Clase', axis=1)
y = df['Clase']

# 3. Definir los parámetros para GridSearchCV
param_grid = {
    'n_neighbors': [1, 3, 7],
    'p': [1, 2]  # 1 para Manhattan
}

# 4. Crear el modelo KNN
model = KNeighborsClassifier()

# 5. Configurar GridSearchCV con validación cruzada de 10 folds
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy')

# 6. Entrenar el modelo
grid_search.fit(X, y)

# 7. Obtener los resultados de la validación cruzada
results = grid_search.cv_results_

# 8. Convertir los resultados a un DataFrame de pandas
results_df = pd.DataFrame(results)

# 9. Obtener el valor de la accuracy para la caja con id 4
split4_test_score = results_df['split4_test_score'].values

print(f"Accuracy en la caja con id 4 para 7 vecinos y distancia Manhattan: {split4_test_score}")

Accuracy en la caja con id 4 para 7 vecinos y distancia Manhattan: [0.92307692 0.84615385 1.         1.         1.         1.        ]


**ChatGPT Solución**

In [69]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Cargar los datos
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/datos_entrenamiento_ssii (1).csv')

# Separar características (X) y etiquetas (y)
X = df.drop("Clase", axis=1)  # Cambia 'Clase' por el nombre real de la columna objetivo
y = df["Clase"]

# Escalar características (opcional, útil si las variables tienen escalas distintas)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definir los parámetros para GridSearchCV
param_grid = {
    "n_neighbors": [3, 7, 11],  # Vecinos
    "metric": ["manhattan", "euclidean"],  # Distancias
}

# Crear el modelo KNN
knn = KNeighborsClassifier()

# Configurar GridSearchCV con validación cruzada de 10 folds
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring="accuracy", return_train_score=True)
grid_search.fit(X_scaled, y)

# Convertir resultados a DataFrame
results = pd.DataFrame(grid_search.cv_results_)

# Crear una tabla con las métricas principales
summary_table = results[[
    "param_n_neighbors", "param_metric", "mean_test_score", "std_test_score"
]].sort_values(by="mean_test_score", ascending=False)

# Mostrar la tabla general
print("Resultados de GridSearchCV:")
print(summary_table)

# Filtrar resultados del modelo específico: 7 vecinos y distancia Manhattan
specific_model = summary_table[
    (results["param_n_neighbors"] == 7) & (results["param_metric"] == "manhattan")
]

print("\nResultados del modelo con 7 vecinos y distancia Manhattan:")
print(specific_model[["mean_test_score", "std_test_score"]])

# Obtener la accuracy específica para el fold 4
accuracy_caja4 = results.loc[
    (results["param_n_neighbors"] == 7) & (results["param_metric"] == "manhattan"),
    "split4_test_score"
].values[0]

# Mostrar el resultado específico
print(f"\nAccuracy en el fold 4: {accuracy_caja4:.4f}")


Resultados de GridSearchCV:
   param_n_neighbors param_metric  mean_test_score  std_test_score
1                  7    manhattan         0.848077        0.103165
5                 11    euclidean         0.823718        0.155658
2                 11    manhattan         0.815385        0.137598
4                  7    euclidean         0.808333        0.124017
3                  3    euclidean         0.791026        0.142394
0                  3    manhattan         0.761538        0.154959

Resultados del modelo con 7 vecinos y distancia Manhattan:
   mean_test_score  std_test_score
1         0.848077        0.103165

Accuracy en el fold 4: 1.0000


<ipython-input-69-129dfb902da8>:43: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  specific_model = summary_table[
